In [1]:
## Libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


## Plot settings
plt.rc('xtick', direction='in', labelsize=14)
plt.rc('ytick', direction='in', labelsize=14)
plt.rc('axes', labelsize=20, titlesize=22)
plt.rc('legend', fontsize=14)
plt.rc('animation', html='jshtml')
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Computer Modern Roman']
plt.rcParams['mathtext.fontset'] = 'cm'

Here we will generate the impurities for the perturbation of the crystal.
We shall generated one gaussian variant around 0 and one similar but all values are positive.
For comparison we also generate one pure.
In short we solve 3 different Hamiltonians and export the results.

In [2]:
# Read in coordinates
## Read coordinates and indices

grid_coordinates = []
with open('../../../../Grids/AB_grid.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(',')
        point = (float(elements[0]),float(elements[1]))
        grid_coordinates.append(point)
        line = reader.readline()

jump_coordinates = []
with open('../../../../Grids/AB_jump_coordinates.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            temp = ele.replace('(', '').replace(')', '')
            if temp != 'stop':
                x = temp.split(',')[0]
                y = temp.split(',')[1]
                point = (float(x),float(y))
                data_block.append(point)
            else:
                break
        line = reader.readline()
        jump_coordinates.append(data_block)

jump_indices = []
with open('../../../../Grids/AB_jump_indices.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            if ele != 'stop':
                data_block.append(int(ele))
            else:
                break
        line = reader.readline()
        jump_indices.append(data_block)

# For drawing vertices use grid_coordinates
# For drawing lines use jump_coordinates
# For indices use jump_indices

## We now want to scale down the plot so it will fit inside a unit square in the 1st quadrant

# Compare x and y components seperatly to for the maximal distances along each axis
xs = [x for x,y in grid_coordinates]
ys = [y for x,y in grid_coordinates]

xs_length = abs(max(xs)) + abs(min(xs))
ys_length = abs(max(ys)) + abs(min(ys))

# We can now scale all xs and ys to fit inside a unit square
xs_scaled = [i/xs_length for i in xs]
ys_scaled = [i/ys_length for i in ys]
grid_scaled = [(x,y) for x, y in zip(xs_scaled, ys_scaled)]

# Same thing for all jump coodinates
jump_scaled = []
for i in jump_coordinates:
    xs_temp = [j[0]/xs_length for j in i]
    ys_temp = [j[1]/ys_length for j in i]
    jump_temp = [(x,y) for x, y in zip(xs_temp, ys_temp)]
    jump_scaled.append(jump_temp)

In [3]:
# Set the value of the magnetic field
B_full = (2*np.pi)/(0.03825242185233838**2)
B_scale = 0.25

In [4]:
# The pure Hamiltonian
def Hamil_solve(z, w, B):
    # Define now a skeleton of the Hamiltonian 
    dim_H = len(z)
    H = np.zeros((dim_H, dim_H), dtype= np.complex128)
    k_H = [x for x in range(0, dim_H)]
    # Calculate Peierls phase factor according to the 1st artcle 
    
    # Start by finding the tile length
    tile_lens = [np.linalg.norm(np.array(z[0]) - np.array(x)) for x in w[0]]
    tile_lens_sum = np.sum(tile_lens)
    l  = tile_lens_sum/len(tile_lens)
    phi = B * l**2
    phi_0 = 2 * np.pi
    jump_products = []
    for i, j in zip(z, w):
        products = [(i[0]*k[1] - k[0]*i[1]) for k in j]
        jump_products.append(products)
    phase_factors = []
    for i, j in zip(z, jump_products):
        phases = [-(phi/(2 * l**2)) * k for k in j]
        phase_factors.append(phases)

    # Scaling via J, maybe energy factor
    J = 1
    # Fill out the Hamiltonian according to the psi vector, i.e. the basis chosen through generation of the tiling
    for x, y, z in zip(k_H, jump_indices, phase_factors):
        for k, t in zip(y, z):
            H[x, k] = -J * np.exp(1j * t)

    # Now we find the eigenvalues and eigenvectors
    # Note that eigh returns normalized eigenvectors!
    # Note the eigenvectors are arranged in a matrix so eig_vecs[:,i] chooses the i'th column which is the i'th eigenvector
    eig_vals, U = np.linalg.eigh(H)
    eig_vecs = np.array([U[:,i] for i in range(len(U))])
    return eig_vals, eig_vecs

pure_eigenvalues, pure_eigenvectors = Hamil_solve(grid_scaled, jump_scaled, (B_scale)*B_full)

# Save outout
np.savetxt("Eigenvalues/pure.txt", pure_eigenvalues)
np.savetxt("Eigenvectors/pure.txt", pure_eigenvectors)

In [5]:
# Load in Vs from other script and call them perturbs
perturbs = np.loadtxt('Vs_sigma_states_546_547_548.txt')
# Define the scaling of Vs
V_scale = 4.45


# The mixed unpure
def Hamil_solve_perturb_mixed(z, w, B, perturb_vals):
    # Define now a skeleton of the Hamiltonian 
    dim_H = len(z)
    H0 = np.zeros((dim_H, dim_H), dtype= np.complex128)
    k_H = np.arange(0, dim_H)
    # Calculate Peierls phase factor according to the 1st artcle 
    
    # Scaling via J, maybe energy factor
    J = 1

    # Start by finding the tile length
    tile_lens = [np.linalg.norm(np.array(z[0]) - np.array(x)) for x in w[0]]
    tile_lens_sum = np.sum(tile_lens)
    l  = tile_lens_sum/len(tile_lens)
    phi = B * l**2
    phi_0 = 2 * np.pi
    jump_products = []
    for i, j in zip(z, w):
        products = [(i[0]*k[1] - k[0]*i[1]) for k in j]
        jump_products.append(products)
    terms_by_sites = []
    for i, j in zip(z, jump_products):
        phases = [-(phi/(2 * l**2)) * k for k in j]
        terms = [-J * np.exp(1j * t) for t in phases]
        terms_by_sites.append(terms)

    # Fill out the Hamiltonian according to the psi vector, i.e. the basis chosen through generation of the tiling
    for x, y, z in zip(k_H, jump_indices, terms_by_sites):
        for k, t in zip(y, z):
            H0[x][k] = t

    # Add the diagonal perturbation
    H_prime = np.diag(perturb_vals)

    # This is the new hamiltonian we will solve, just as previously
    H = H0 + H_prime

    eig_vals, U = np.linalg.eigh(H)
    eig_vecs = np.array([U[:,i] for i in range(len(U))])
    
    return eig_vals, eig_vecs

unpure_mixed_eigenvalues, unpure_mixed_eigenvectors = Hamil_solve_perturb_mixed(grid_scaled, jump_scaled, (B_scale)*B_full, V_scale*perturbs)

# Save outout
np.savetxt("Eigenvalues/unpure_mixed.txt", unpure_mixed_eigenvalues)
np.savetxt("Eigenvectors/unpure_mixed.txt", unpure_mixed_eigenvectors)

In [6]:
# The mixed unpure
def Hamil_solve_perturb_positive(z, w, B, perturb_vals):
    # Define now a skeleton of the Hamiltonian 
    dim_H = len(z)
    H0 = np.zeros((dim_H, dim_H), dtype= np.complex128)
    k_H = np.arange(0, dim_H)
    # Calculate Peierls phase factor according to the 1st artcle 
    
    # Scaling via J, maybe energy factor
    J = 1

    # Start by finding the tile length
    tile_lens = [np.linalg.norm(np.array(z[0]) - np.array(x)) for x in w[0]]
    tile_lens_sum = np.sum(tile_lens)
    l  = tile_lens_sum/len(tile_lens)
    phi = B * l**2
    phi_0 = 2 * np.pi
    jump_products = []
    for i, j in zip(z, w):
        products = [(i[0]*k[1] - k[0]*i[1]) for k in j]
        jump_products.append(products)
    terms_by_sites = []
    for i, j in zip(z, jump_products):
        phases = [-(phi/(2 * l**2)) * k for k in j]
        terms = [-J * np.exp(1j * t) for t in phases]
        terms_by_sites.append(terms)

    # Fill out the Hamiltonian according to the psi vector, i.e. the basis chosen through generation of the tiling
    for x, y, z in zip(k_H, jump_indices, terms_by_sites):
        for k, t in zip(y, z):
            H0[x][k] = t

    # Add the diagonal perturbation
    perturbs = [abs(x) for x in perturb_vals]
    H_prime = np.diag(perturbs)

    # This is the new hamiltonian we will solve, just as previously
    H = H0 + H_prime

    eig_vals, U = np.linalg.eigh(H)
    eig_vecs = np.array([U[:,i] for i in range(len(U))])
    
    return eig_vals, eig_vecs

unpure_positive_eigenvalues, unpure_positive_eigenvectors = Hamil_solve_perturb_positive(grid_scaled, jump_scaled, (B_scale)*B_full, V_scale*perturbs)

# Now we add the value of V scaling to the first entry of the eigenvalues list to save it for later
un_pur_evals = np.insert(unpure_positive_eigenvalues, 0, V_scale)

# Save outout
np.savetxt("Eigenvalues/unpure_positive.txt", un_pur_evals)
np.savetxt("Eigenvectors/unpure_positive.txt", unpure_positive_eigenvectors)